# Лабораторная работа №5 (Пономарева А.Ю., группа БПМ-151)
## РЕШЕНИЕ  СИСТЕМ  ЛИНЕЙНЫХ АЛГЕБРАИЧЕСКИХ УРАВНЕНИЙ ИТЕРАЦИОННЫМИ МЕТОДАМИ 
### Вариант 23

## Задача 5.1

Дана  система уравнений  $Ax = b$. Найти решение  системы  с помощью метода Гаусса. Выполнить $10$ итераций по методу Зейделя. Принимая решение,  полученное с помощью метода Гаусса  за точное, найти величину абсолютной погрешности итерационного решения. 

$A =  \begin{bmatrix}
   118.8 & -14 & -5 & -89.1 \\    
   -59.4 & 194 & 5 & 128.7 \\
   148.5 & 12 & -310 & 148.5 \\
   0 & 18.5 & 90 & -108.9    
   \end{bmatrix}
$

$ b =\begin{bmatrix}
451.5 \\
-1158.3 \\
5700\\
-2060.7
\end{bmatrix}
$

In [1]:
import numpy as np
from math import sqrt
from numpy.linalg import solve, norm
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
A = np.array([[118.8, -14, -5, -89.1], 
              [-59.4, 194, 5, 128.7], 
              [148.5, 12, -310, 148.5], 
              [0, 18.5, 90, -108.9]])
b = np.array([[451.5],
              [-1158.3],
              [5700],
              [-2060.7]])

In [3]:
A

array([[ 118.8,  -14. ,   -5. ,  -89.1],
       [ -59.4,  194. ,    5. ,  128.7],
       [ 148.5,   12. , -310. ,  148.5],
       [   0. ,   18.5,   90. , -108.9]])

In [4]:
b

array([[  451.5],
       [-1158.3],
       [ 5700. ],
       [-2060.7]])

С помощью встроенной в $numpy$ функции $solve$ найдем истинное решение

In [5]:
x_true = solve(A, b)

In [6]:
x_true

array([[ 9.23232323],
       [-9.        ],
       [-9.9       ],
       [ 9.21212121]])

**Преобразуем систему $Ax=b$ к виду $x=Bx+c$, удобному для итераций.** <br/>
Как говорит нам учебник Амосова: <br/>
"Для того чтобы применить метод простой итерации к решению СЛАУ $ Ax = b$ с квадратной невырожденной матрицей $A$ необходимо преобразовать систему к виду: $$\large x = Bx + c\qquad (1)$$
где $\large B$ - квадратная матрица с элементами $\large b_{i, j}, i, j = 1, ... , n$,

$\large c$ - вектор-столбец с элементами $\large c_{i}, i = 1, ..., n$

В развернутой форме система (1) примет вид:

$$\large x_{1} = b_{1,1}x_{1} + b_{1,2}x_{2} + b_{1,3}x_{3} + ... + b_{1, n}x_{n} + c_{1},$$$$\large x_{2} = b_{2,1}x_{1} + b_{2,2}x_{2} + b_{2,3}x_{3} + ... + b_{2, n}x_{n} + c_{2},$$$$\large \dotsb \quad \dotsb \quad \dotsb \quad \dotsb \quad \dotsb$$$$\large x_{n} = b_{n,1}x_{1} + b_{n,2}x_{2} + b_{n,3}x_{3} + ... + b_{n, n}x_{n} + c_{n},$$
Вообще говоря данная операция приведения к виду удобному для итераций не является простой и требует специальных знаний о специфике системы.
Самый простой способ состоит в следующем:

$\large x_{1} = a_{1,1}^{-1}(b_{1} - a_{1,2}x_{1} - a_{1,3}x_{2} - a_{1,3}x_{3} - ... - a_{1,n}x_{n})$ - из первого уравнения системы выражаем $x_{1}$,

$\large x_{2} = a_{2,2}^{-1}(b_{2} - a_{2,1}x_{1} - a_{2,3}x_{2} - a_{3,3}x_{3} - ... - a_{3,n}x_{n})$ - из первого уравнения системы выражаем $x_{3}$,

Продолжаем выражать корни и получаем систему:

$$\large x_{1} = \qquad\quad b_{1,2}x_{2} + b_{1,3}x_{3} + \dotsb + b_{1, n}x_{n} + c_{1},$$$$\large x_{2} = b_{2,1}x_{1} + \qquad\quad b_{2,3}x_{3} + \dotsb + b_{2, n}x_{n} + c_{2},$$$$\large x_{3} = b_{3,1}x_{1} + b_{3,2}x_{2}  \qquad\quad + \dotsb + b_{2, n}x_{n} + c_{2},$$
$\qquad\qquad \qquad\qquad \qquad\qquad \qquad\qquad  \dotsb \quad \dotsb \quad \dotsb \quad \dotsb \quad \dotsb \qquad\qquad \qquad\qquad (2) $ $$\large x_{n} = b_{n,1}x_{1} + b_{n,2}x_{2} + b_{n,3}x_{3} + \dotsb \qquad\quad +c_{n},$$ в которой на главной диагонали стоят нулевые элементы.Остальные выражаются по формулам: $\large b_{i, j} = \frac{-a_{i, j}}{a_{i, i}}, c_{i} = \frac{b_{i}}{a_{i, i}}, \quad i,j = 1, ..., n, i\neq j  \quad\quad (3)$

Конечно для выполнения данного преобразования необходимо, чтобы диагональные элементы матрицы $А$ были не нулевыми".

Зададим функцию, которая раскладывает матрицу $A$ c ненулевыми диагональными элементами на верхнетреугольную матрицу $B_2$ и нижнетреугольную матрицу $B_1$.

In [7]:
def transform(A, b):
    B1, B2 = np.zeros_like(A), np.zeros_like(A)
    c = np.zeros_like(b)
    for i in range(len(A)):
        B1[i, :i] = -A[i, :i]/A[i, i]
        B2[i, i+1:] = -A[i, i+1:]/A[i, i]
        c[i] = b[i]/A[i, i]
    return B1, B2, c

Приведем данную нам систему $Ax=b$ к итерируемому виду с помощью написанной функции $transform$. 

In [8]:
B1, B2, c = transform(A, b)

In [9]:
B1

array([[0.        , 0.        , 0.        , 0.        ],
       [0.30618557, 0.        , 0.        , 0.        ],
       [0.47903226, 0.03870968, 0.        , 0.        ],
       [0.        , 0.16988062, 0.82644628, 0.        ]])

In [10]:
B2

array([[ 0.        ,  0.11784512,  0.04208754,  0.75      ],
       [ 0.        ,  0.        , -0.0257732 , -0.66340206],
       [ 0.        ,  0.        ,  0.        ,  0.47903226],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [11]:
c

array([[  3.80050505],
       [ -5.97061856],
       [-18.38709677],
       [ 18.92286501]])

Теперь следует проверить выполнение достаточного условия сходимости итерационных методов: 
<br>$||B||_{\infty} < 1$</br>

In [12]:
B_norm = norm(B1+B2, ord=np.inf)
print(B_norm < 1)

True


Зададим функцию, реализующую метод Гаусса-Зейделя. 
Внтури нее будем вызывать уже написанную функцию $transform(A,b)$: так удобнее работать с изначальной системой.

In [31]:
def gase(A, b, x0, n=100, eps=0):
    B1, B2, c = transform(A, b) # приводим систему к итерационному виду
    B = B1 + B2
    count=1 # счита итерации
    
    x = np.copy(x0)
    eps2 = eps*(1 - np.linalg.norm(B1+B2, np.inf))/np.linalg.norm(B2, np.inf)

    for k in range(n):
        x_new = np.copy(x)
        count+=1
        for i in range(B.shape[0]):
            s1 = np.dot(B[i, :i], x_new[:i])
            s2 = np.dot(B[i, i + 1:], x[i + 1:])
            x_new[i] = c[i] + s1 + s2
            converge = np.linalg.norm((x_new - x), ord=2) < eps2
        if converge:
                print ('кол-во итераций -',count)
                return x_new
        x = x_new 
    print ('кол-во итераций -',count)
    return x

In [14]:
x0 = np.zeros_like(c)
x_10 = gase(A, b, x0, 10, eps=0)

кол-во итераций - 11


Принимая решение, полученное методом $linalg.solve(A, b)$, за точное, найдем величину абсолютной погрешности итерационного решения (используя норму $||\cdot||_{\infty}$). 

In [36]:
print('Решение встроенным методом (истинное):\n {0}'.format(x_true.flatten().T))
print('Решение методом Гаусса-Зейделя за 11 итераций для x0 ={0}:\n {1}'.format(x0.tolist(), x_10.flatten().T))
print('Ошибка:\n{0}'.format(norm((x_true-x_10), ord=np.inf)))

Решение встроенным методом (истинное):
 [ 9.23232323 -9.         -9.9         9.21212121]
Решение методом Гаусса-Зейделя за 11 итераций для x0 =[[0.0], [0.0], [0.0], [0.0]]:
 [ 9.18744407 -8.96957409 -9.95058966  9.17548035]
Ошибка:
0.050589661873310376


Теперь нам нужно взять *любое другое* начальное приближение и объяснить полученные результаты. 

In [30]:
x0 = np.ones_like(b)
x_1 = gase(A, b, x0, 10)
x_1

кол-во итераций - 11


array([[ 9.19369519],
       [-8.97381205],
       [-9.94354313],
       [ 9.18058398]])

In [35]:
print('Решение встроенным методом (истинное):\n {0}'.format(x_true.flatten().T))
print('Решение методом Гаусса-Зейделя за 11 итераций для x0 ={0}:\n {1}'.format(x0.tolist(), x_1.flatten().T))
print('Ошибка:\n{0}'.format(norm((x_true-x_1), ord=np.inf)))

Решение встроенным методом (истинное):
 [ 9.23232323 -9.         -9.9         9.21212121]
Решение методом Гаусса-Зейделя за 11 итераций для x0 =[[0.0], [0.0], [0.0], [0.0]]:
 [ 9.19369519 -8.97381205 -9.94354313  9.18058398]
Ошибка:
0.04354313234117413


In [18]:
x0 = np.array([[9.19369519] ,[-8.97381205], [-9.94354313] , [9.18058398]])
x_close = gase(A, b, x0, 10)

кол-во итераций - 11


In [34]:
print('Решение встроенным методом (истинное):\n {0}'.format(x_true.flatten().T))
print('Решение методом Гаусса-Зейделя за 11 итераций для x0 ={0}:\n {1}'.format(x0.tolist(), x_close.flatten().T))
print('Ошибка:\n{0}'.format(norm((x_true-x_close), ord=np.inf)))

Решение встроенным методом (истинное):
 [ 9.23232323 -9.         -9.9         9.21212121]
Решение методом Гаусса-Зейделя за 11 итераций для x0 =[[0.0], [0.0], [0.0], [0.0]]:
 [ 9.23215718 -8.99988742 -9.90018718  9.21198564]
Ошибка:
0.0001871812232678849


Ошибка меньше, т.к. мы взяли достаточно точное приближение к истинному решению системы $x_true$, и за 11 итераций сильнее приблизились к истинному ответу.

## Задача 5.2
Для системы уравнений $\large Ax=b$ из задачи 1 найти решение по методу Зейделя с точностью $\large \varepsilon = 10^{-6}$, взяв *любое* начальное приближение. Подсчитать количество итераций до сходимости.

In [32]:
x0 = np.zeros_like(c)
x_eps = gase(A, b, x0, eps=1e-6)
x_eps

кол-во итераций - 42


array([[ 9.23232323],
       [-9.        ],
       [-9.9       ],
       [ 9.21212121]])

In [33]:
print('Решение встроенным методом (истинное):\n {0}'.format(x_true.flatten().T))
print('Решение методом Гаусса-Зейделя с начальным приближением x_ones = {0} и точностью epsilon = 10^(-6):\n {1}'.format(x0.tolist(), x_eps.flatten().T))
print('Ошибка:\n{0}'.format(norm((x_true-x_eps), ord=np.inf)))

Решение встроенным методом (истинное):
 [ 9.23232323 -9.         -9.9         9.21212121]
Решение методом Гаусса-Зейделя с начальным приближением x_ones = [[0.0], [0.0], [0.0], [0.0]] и точностью epsilon = 10^(-6):
 [ 9.23232323 -9.         -9.9         9.21212121]
Ошибка:
2.3303581286882036e-09
